<a href="https://colab.research.google.com/github/Sibin-S-S/Sibin_DSA_Pathshaala/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, render_template
import pandas as pd

# Initialize the Flask application
app = Flask(__name__)

# --- LOAD FILE HERE ---
# Make sure netflix_cleaned.csv is in the same folder as this app.py
df = pd.read_csv("netflix_cleaned.csv")


for col in ["type", "country", "cast"]:
    if col in df.columns:
        df[col] = df[col].fillna(f"Unknown {col.capitalize()}").astype(str)
    else:
        df[col] = f"Unknown {col.capitalize()}"


def explode_cast(df_in: pd.DataFrame) -> pd.DataFrame:
    """Split comma-separated cast into one row per actor."""
    temp = df_in.copy()
    temp["cast"] = (
        temp["cast"]
        .astype(str)
        .str.split(",")
        .apply(lambda lst: [x.strip() for x in lst] if isinstance(lst, list) else [])
    )
    temp = temp.explode("cast")
    temp = temp.rename(columns={"cast": "actor"})
    return temp


@app.route('/', methods=['GET', 'POST'])
def home():
    """
    Renders the dashboard page (index.html).
    Allows filtering by content type (Movie / TV Show / All) and
    displays aggregated tables for actors and countries.
    """


    selected_type = request.form.get("content_type", "All")


    if selected_type == "All":
        df_filtered = df.copy()
    else:
        df_filtered = df[df["type"] == selected_type].copy()


    # Top 10 actors: {"actor": <name>, "count": <number>}
    df_actors = explode_cast(df_filtered)
    top_actors = (
        df_actors["actor"]
        .value_counts()
        .head(10)
        .reset_index()
        .rename(columns={"index": "actor", "actor": "count"})
        .to_dict(orient="records")
    )

    # Top 10 countries: {"country": <name>, "count": <number>}
    top_countries = (
        df_filtered["country"]
        .value_counts()
        .head(10)
        .reset_index()
        .rename(columns={"index": "country", "country": "count"})
        .to_dict(orient="records")
    )

    return render_template(
        'index.html',
        selected_type=selected_type,
        top_actors=top_actors,
        top_countries=top_countries,
    )



@app.route('/predict', methods=['POST'])
def predict():

    return render_template(
        'index.html',
        selected_type="All",
        top_actors=[],
        top_countries=[],
    )


if __name__ == "__main__":
    app.run(debug=True)